In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor  # Geändert zu einem komplexeren Modelltyp
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
# Ersetze 'dein_dateipfad' mit dem Pfad zu deinen CSV-Dateien
abgebrochene_wohnungen_path = '/Users/mariusaffolter/Documents/200_Studium/220_Semester/24FS/Einsatz Geodaten Marketing/Project/apartment_analysis_egm/abgebrochene_wohnungen.csv'
bestehende_wohnungen_path = '/Users/mariusaffolter/Documents/200_Studium/220_Semester/24FS/Einsatz Geodaten Marketing/Project/apartment_analysis_egm/bestehende_wohnungen.csv'

try:
    abgebrochene_wohnungen = pd.read_csv(abgebrochene_wohnungen_path)
    print("Columns in abgebrochene_wohnungen:", abgebrochene_wohnungen.columns)
    print("Sample data from abgebrochene_wohnungen:", abgebrochene_wohnungen.head())
except Exception as e:
    print("Error loading abgebrochene_wohnungen:", e)

try:
    bestehende_wohnungen = pd.read_csv(bestehende_wohnungen_path)
    print("Columns in bestehende_wohnungen:", bestehende_wohnungen.columns)
    print("Sample data from bestehende_wohnungen:", bestehende_wohnungen.head())
except Exception as e:
    print("Error loading bestehende_wohnungen:", e)

Columns in abgebrochene_wohnungen: Index(['EGID', 'Official Building No', 'Building Status', 'Building Category',
       'Building Class', 'Date of Construction', 'Year of Demolition',
       'Surface Area of Building', 'Number of Floors', 'Heating Device Info',
       'Record Modification Info'],
      dtype='object')
Sample data from abgebrochene_wohnungen:      EGID Official Building No  Building Status  Building Category  \
0  140078                  854             1007               1020   
1  140604                  868             1007               1020   
2  141094                 7597             1007               1020   
3  141126                 1920             1007               1020   
4  141156                 7057             1007               1020   

   Building Class Date of Construction  Year of Demolition  \
0            1121                 1837                2007   
1            1121                 1838                2009   
2            1110              

In [3]:
# Daten laden
abgebrochene_wohnungen = pd.read_csv('/Users/mariusaffolter/Documents/200_Studium/220_Semester/24FS/Einsatz Geodaten Marketing/Project/apartment_analysis_egm/abgebrochene_wohnungen.csv')

# Bereinigung der Spalte 'Date of Construction'
abgebrochene_wohnungen['Date of Construction'] = abgebrochene_wohnungen['Date of Construction'].astype(str)  # Erzwinge den Typ zu String
abgebrochene_wohnungen['Date of Construction'] = abgebrochene_wohnungen['Date of Construction'].apply(lambda x: int(x.split('-')[0]) if '-' in x else float('nan'))

# Entferne Zeilen mit NaN-Werten in 'Date of Construction'
abgebrochene_wohnungen = abgebrochene_wohnungen.dropna(subset=['Date of Construction'])

# Kategorische Variablen One-Hot-Encoden
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
categorical_columns = ['Building Status', 'Building Category', 'Building Class']
encoded_categories = encoder.fit_transform(abgebrochene_wohnungen[categorical_columns])
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out())

# Alter des Gebäudes als neues Feature hinzufügen
CURRENT_YEAR = 2024  # Setze das aktuelle Jahr
abgebrochene_wohnungen['Building Age'] = CURRENT_YEAR - abgebrochene_wohnungen['Date of Construction'].astype(int)

# Numerische Daten kombinieren
numeric_columns = ['Date of Construction', 'Surface Area of Building', 'Number of Floors', 'Building Age']  # 'Building Age' hinzugefügt
numeric_data = abgebrochene_wohnungen[numeric_columns].reset_index(drop=True)
combined_data = pd.concat([encoded_df, numeric_data], axis=1)

# Zielvariable
y = abgebrochene_wohnungen['Year of Demolition']

# Daten aufteilen
X_train, X_test, y_train, y_test = train_test_split(combined_data, y, test_size=0.2, random_state=42)

# Gradient Boosting Regressionsmodell
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

# Vorhersage und Bewertung
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print('MSE:', mse)
print('Predictions:', y_pred[:5])

MSE: 9.78465166842956
Predictions: [2022.91888052 2017.08415259 2022.91888052 2022.58660189 2022.22802192]


/opt/anaconda3/envs/spenv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
# Daten für bestehende Wohnungen laden
bestehende_wohnungen = pd.read_csv('/Users/mariusaffolter/Documents/200_Studium/220_Semester/24FS/Einsatz Geodaten Marketing/Project/apartment_analysis_egm/bestehende_wohnungen.csv')

# Bereinigung der Spalte 'Date of Construction' wie beim Trainingsdatensatz
bestehende_wohnungen['Date of Construction'] = bestehende_wohnungen['Date of Construction'].astype(str)
bestehende_wohnungen['Date of Construction'] = bestehende_wohnungen['Date of Construction'].apply(lambda x: int(x.split('-')[0]) if '-' in x else float('nan'))
bestehende_wohnungen = bestehende_wohnungen.dropna(subset=['Date of Construction'])

# Alter des Gebäudes als neues Feature hinzufügen (entsprechend dem Trainingsdatensatz)
CURRENT_YEAR = 2024
bestehende_wohnungen['Building Age'] = CURRENT_YEAR - bestehende_wohnungen['Date of Construction'].astype(int)

# One-Hot-Encoding der kategorischen Variablen
# Achtung: Verwende denselben Encoder, der für das Training verwendet wurde, um Konsistenz zu gewährleisten
encoded_categories_bestehende = encoder.transform(bestehende_wohnungen[categorical_columns])
encoded_df_bestehende = pd.DataFrame(encoded_categories_bestehende, columns=encoder.get_feature_names_out())

# Numerische Daten kombinieren
numeric_columns = ['Date of Construction', 'Surface Area of Building', 'Number of Floors', 'Building Age']  # Stelle sicher, dass 'Building Age' enthalten ist
numeric_data_bestehende = bestehende_wohnungen[numeric_columns].reset_index(drop=True)
combined_data_bestehende = pd.concat([encoded_df_bestehende, numeric_data_bestehende], axis=1)

# Vorhersagen mit dem trainierten Modell erstellen
predictions_bestehende = model.predict(combined_data_bestehende)

# Ergebnisse in einen DataFrame schreiben und als CSV speichern
bestehende_wohnungen['Predicted Year of Demolition'] = predictions_bestehende
bestehende_wohnungen.to_csv('/Users/mariusaffolter/Documents/200_Studium/220_Semester/24FS/Einsatz Geodaten Marketing/Project/apartment_analysis_egm/large_dataset_predicted_bestehende_wohnungen.csv', index=False)

print("Vorhersagen wurden gespeichert.")

Vorhersagen wurden gespeichert.
